In [1]:
file = open('input.txt', 'r')
input = file.readlines()
file.close()

In [2]:
#dictionary with coordinates, labels and connections
pipe_maze = {}

In [3]:
#dictionary with path lengths
path_length = {}

In [4]:
#list with enclosed tiles
enclosed_tiles = []

In [5]:
def print_maze(maze, distances=False, enclosed=False):
    for i in range(len(input)):
        line = ""
        for j in range(len(input[0].strip())):
            if (i,j) in enclosed_tiles:
                line += 'I'
            elif (i,j) not in maze.keys():
                line += '.'
            else:
                if distances:
                    line += str(path_length[(i,j)])
                else:
                    line += maze[(i,j)][0]
        print(line)

In [6]:
#build pipe maze
starting_coord = None
for i, line in enumerate(input):
    for j, character in enumerate(line.strip()):
        coord = (i,j)
        #print(coord, character)
        if character == '.':
            continue
        elif character == 'F':
            connection1 = (coord[0]+1,coord[1])
            connection2 = (coord[0],coord[1]+1)
        elif character == '-':
            connection1 = (coord[0],coord[1]-1)
            connection2 = (coord[0],coord[1]+1)        
        elif character == '7':
            connection1 = (coord[0],coord[1]-1)
            connection2 = (coord[0]+1,coord[1])
        elif character == '|':
            connection1 = (coord[0]+1,coord[1])
            connection2 = (coord[0]-1,coord[1])                   
        elif character == 'L':
            connection1 = (coord[0]-1,coord[1])
            connection2 = (coord[0],coord[1]+1)   
        elif character == 'J':
            connection1 = (coord[0]-1,coord[1])
            connection2 = (coord[0],coord[1]-1)
        elif character == 'S':
            starting_coord = coord
            connection1 = None
            connection2 = None
        else:
            continue
        pipe_maze[coord] = (character, connection1, connection2)

In [7]:
#set starting point connections
if not starting_coord == None:
    connections = []
    #check left
    left = (starting_coord[0],starting_coord[1]-1)
    if left in pipe_maze.keys():
        if pipe_maze[left][0] in ['L','-','F']:
            connections.append(left)
    #check right
    right = (starting_coord[0],starting_coord[1]+1)
    if right in pipe_maze.keys():
        if pipe_maze[right][0] in ['J','-','7']:
            connections.append(right)    
    #check up
    up = (starting_coord[0]-1,starting_coord[1])
    if up in pipe_maze.keys():
        if pipe_maze[up][0] in ['7','|', 'F']:
            connections.append(up)
    #check down
    down = (starting_coord[0]+1,starting_coord[1])
    if down in pipe_maze.keys():
        if pipe_maze[down][0] in ['J','|', 'L']:
            connections.append(down)
    pipe_maze[starting_coord] = ('S',connections[0], connections[1])


In [8]:
print_maze(pipe_maze)

-L7F7-J77|-FF7F-.|-JF77FF-FLJF--F--77-77-F|-F77.FF|7-F|-L-LL.L.77F7F-J.F-LFJ7-F|-F7J-F|.F7.FF-7.FJ7FL7F|-F-F---F-FL-F-7J7F.F77.F77FJ.77.L.7J
J.||L-7FL|.|LF|.7JF-LJF-J7J.|7.F-7FF7F|FF-F-JJ-77J|F.|J-|.|.FL7--F77LJF|-LLL-7J||LJ7-JJ.LJL-J|F77|F-7-J..-F-JLFFFJJ||7|7-F-J7--L7-FJLJLJ-L..
|-|-JLLJ|LF|-J|FJF7.L7-JFF.-|7L7FLJ|FF-7-J||7L7F7.|.--7-LF7F|F7JF||LLF7L7.7JJ.-FFJ|7||.LJ7.LF-|.F7J.|JF|7J|77.L7JJ7|.F77-J7LJFJ-7.L7L7JJ7-LF
FF7LF7.FLFL.LL-7|-J.L---|F-.-7JFF-J7FF7JJFFLJF-7F7F7F7.FJF-77JL77.J.|||FJ-7-F7.L7-7L-J-|F-7J|FJ--7-F|F-L.--JL|L|77|JF||L-J7--77--J77F|L.|.|J
FLJ7FF-F7LLF.FJ7..-L-JJ7|J.FLL-FF7L-LJL|F-LJJJLFJLJL7F7|F|FJ7LFFJFLFFFJJJ7L-JFJJLL|7|---|-7-J7|.-|LLJ7|LF|J7F7-J.LJ7-LJFLL-7.FL7.LF7||.FJ7JF
L.L|L|7J77LLFL7L--L-7JJJLJ-7J7FJLF.|7F-F-FJJ|.-L--7FJ||F-J|JJ7.F7J|FJ|JJL77FL-.|LF|.F7-LLJ||FF--7F-J-LJ.FJ.LJJ||-.FF7-LL7FL.7JJF7.JL-J-J-L-|
|7JL7LJ-F777JF-J.FJFL77F||...-7-FJ7F|J-|FJL|7FJLF7|L7|||F7|F7-FJ|L|JF7|F-FJ|.|.7-FJ-|J-.L-LJJJJFJ|L77|-FJFFJJFF|-L7.--.L-F.L7|F7--7F||L7|LFJ
||LF|---FJ||J

In [9]:
#traverse maze starting with 'S' (starting_coord)
#end check for free, eclosed spaces right from traversal direction
#Attention: loop has to be traversed counter clock-wise!
def take_path(source, dest, enclosed = False):
    distance = 1
    while True:
        if enclosed:
            #check right hand tiles in direction of traversal
            #if dest is a corner pipe, several directions have to be checked
            direction = (source[0]-dest[0],source[1]-dest[1])
            #print(direction)
            neighbors = []
            neighbors2 = []
            neighbors.append(dest)
            neighbors2.append(dest)
            additional_neighbor = dest
            while True:
                if direction == (0,-1):
                    #right -> check downwards
                    right_neighbor = (neighbors[-1][0]+1, neighbors[-1][1])
                    #if J also check rightwards
                    if pipe_maze[dest][0] == 'J':
                        additional_neighbor = (neighbors2[-1][0], neighbors2[-1][1]+1)
                elif direction == (-1,0):
                    #down -> check leftwards
                    right_neighbor = (neighbors[-1][0], neighbors[-1][1]-1)
                    #if L also check downwards
                    if pipe_maze[dest][0] == 'L':
                        additional_neighbor = (neighbors2[-1][0]+1, neighbors2[-1][1])
                elif direction == (0,1):
                    #left -> check upwards
                    right_neighbor = (neighbors[-1][0]-1, neighbors[-1][1])
                    #if F also check leftwards
                    if pipe_maze[dest][0] == 'F':
                        additional_neighbor = (neighbors2[-1][0], neighbors2[-1][1]-1)
                elif direction == (1,0):
                    #up -> check rightwards
                    right_neighbor = (neighbors[-1][0], neighbors[-1][1]+1)
                    #if 7 also check upwards
                    if pipe_maze[dest][0] == '7':
                        additional_neighbor = (neighbors2[-1][0]-1, neighbors2[-1][1])
                if right_neighbor in pipe_maze.keys() and additional_neighbor in pipe_maze.keys():
                    break
                if right_neighbor not in pipe_maze.keys():
                    neighbors.append(right_neighbor)
                    #print(right_neighbor)
                if additional_neighbor not in pipe_maze.keys():
                    neighbors2.append(additional_neighbor)
                    #print(additional_neighbor)

            for tile in neighbors:
                if tile not in enclosed_tiles and tile not in pipe_maze.keys():
                    enclosed_tiles.append(tile)
            for tile in neighbors2:
                if tile not in enclosed_tiles and tile not in pipe_maze.keys():
                    enclosed_tiles.append(tile)

        
        if dest in path_length.keys(): 
            if distance >= path_length[dest]:
                return
        path_length[dest] = distance
        if pipe_maze[dest][1] != source:
            source = dest
            dest = pipe_maze[dest][1]
        else:
            source = dest
            dest = pipe_maze[dest][2]
        distance += 1

In [10]:
#remove non main-loop pipes
path_length[starting_coord] = 0
#direction must be counter clock-wise: trial and error
take_path(starting_coord, pipe_maze[starting_coord][1])


In [11]:
keys = list(pipe_maze.keys())
for key in keys:
    if key not in path_length.keys():
        pipe_maze.pop(key)

In [12]:
print_maze(pipe_maze, False, True)

............................................................................................................................................
............................................................................................................................................
............................................................................................................................................
................................................F7F7.....F-7................................................................................
...............................................FJLJL7F7..|FJ................................................................................
...............................................L--7FJ||F-J|....F7...........................................................................
................................................F7|L7|||F7|F7.FJ|...F7......................................................................
.............

In [13]:
path_length = {}
path_length[starting_coord] = 0
#direction must be counter clock-wise: trial and error
take_path(starting_coord, pipe_maze[starting_coord][1], True)

In [14]:
print_maze(pipe_maze, False, True)

............................................................................................................................................
............................................................................................................................................
............................................................................................................................................
................................................F7F7.....F-7................................................................................
...............................................FJLJL7F7..|FJ................................................................................
...............................................L--7FJ||F-J|....F7...........................................................................
................................................F7|L7|||F7|F7.FJ|...F7......................................................................
.............

In [15]:
print(len(enclosed_tiles))

459
